In [1]:
!pip install elasticsearch
!pip install csv2es
!pip install requests

  Using cached elasticsearch-7.10.1-py2.py3-none-any.whl (322 kB)
     |████████████████████████████████| 137 kB 2.3 MB/s eta 0:00:01
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

pyelasticsearch 1.2.3 requires elasticsearch<2.0.0,>=1.3.0, but you'll have elasticsearch 7.10.1 which is incompatible.
Processing /home/aluno/.cache/pip/wheels/f0/ba/36/52ee8986e4a509ea1b0b79cb9a595c0c0cc8f124686f89702e/csv2es-1.0.1-py3-none-any.whl
Processing /home/aluno/.cache/pip/wheels/83/b4/e7/a4bb7ef2f9ed2d868f85f3362cfcb1c5d01f4e2e91c1e40972/urllib3-1.10.2-py3-none-any.whl
  Using cached elasticsearch-1.4.0-py2.py3-none-any.whl (56 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.3
    Uninstalling urllib3-1.26.3:
      Succ

In [2]:
from elasticsearch import helpers, Elasticsearch
import elasticsearch
import csv
import csv2es

In [3]:
es = Elasticsearch()

In [4]:
df = spark.read.csv('DNPBA2017.csv', header=True)

In [5]:
#df = df.select(['NUMERODN', 'CODESTAB', 'IDADEMAE', 'ESCMAEAGR1']).limit(20).toPandas()
df = df.toPandas()

In [6]:
df.to_csv('DNPBA2017_es.csv', index=False)

### indexng

In [7]:
with open('DNPBA2017_es.csv') as f:
    reader = csv.DictReader(f)
    helpers.bulk(es, reader, index='IndexGr03', doc_type='my-type')
    
# ler indices no linux com: curl 'localhost:9200/_cat/indices?v'

POST http://localhost:9200/IndexGr03/my-type/_bulk [status:N/A request:0.002s]
Traceback (most recent call last):
  File "/home/aluno/anaconda3/lib/python3.8/site-packages/urllib3/connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "/home/aluno/anaconda3/lib/python3.8/site-packages/urllib3/util/connection.py", line 96, in create_connection
    raise err
  File "/home/aluno/anaconda3/lib/python3.8/site-packages/urllib3/util/connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/aluno/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/http_urllib3.py", line 71, in perform_request
    response = self.pool.urlopen(method, url, body, retries=False, headers=self.headers, **kw)
  File "/home/aluno/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", l

POST http://localhost:9200/IndexGr03/my-type/_bulk [status:N/A request:0.001s]
Traceback (most recent call last):
  File "/home/aluno/anaconda3/lib/python3.8/site-packages/urllib3/connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "/home/aluno/anaconda3/lib/python3.8/site-packages/urllib3/util/connection.py", line 96, in create_connection
    raise err
  File "/home/aluno/anaconda3/lib/python3.8/site-packages/urllib3/util/connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/aluno/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/http_urllib3.py", line 71, in perform_request
    response = self.pool.urlopen(method, url, body, retries=False, headers=self.headers, **kw)
  File "/home/aluno/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", l

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7efc23fb7fa0>: Failed to establish a new connection: [Errno 111] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7efc23fb7fa0>: Failed to establish a new connection: [Errno 111] Connection refused)

### buscando termo

In [66]:
content = {
    "query": {
        "term": {
            "CODESTAB": "2786095"
         }
     },
}

In [67]:
res = es.search(index="my-index", body=content)

In [68]:
res

{u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 1, u'total': 1},
 u'hits': {u'hits': [{u'_id': u'H8atyW8BbundlQk4fUZA',
    u'_index': u'my-index',
    u'_score': 2.6390574,
    u'_source': {u'CODESTAB': u'2786095',
     u'ESCMAEAGR1': u'01',
     u'IDADEMAE': u'25',
     u'NUMERODN': u'72390242'},
    u'_type': u'my-type'}],
  u'max_score': 2.6390574,
  u'total': {u'relation': u'eq', u'value': 1}},
 u'timed_out': False,
 u'took': 1}

### buscando um intervalo

In [71]:
content = {
    "query": {
        "range": {
            "IDADEMAE": {
                "gte": 20,
                "lt": 30
            }
        }
    }
}

In [72]:
res = es.search(index="my-index", body=content)

In [73]:
res

{u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 1, u'total': 1},
 u'hits': {u'hits': [{u'_id': u'H8atyW8BbundlQk4fUZA',
    u'_index': u'my-index',
    u'_score': 1.0,
    u'_source': {u'CODESTAB': u'2786095',
     u'ESCMAEAGR1': u'01',
     u'IDADEMAE': u'25',
     u'NUMERODN': u'72390242'},
    u'_type': u'my-type'},
   {u'_id': u'IMatyW8BbundlQk4fUZB',
    u'_index': u'my-index',
    u'_score': 1.0,
    u'_source': {u'CODESTAB': u'NA',
     u'ESCMAEAGR1': u'02',
     u'IDADEMAE': u'20',
     u'NUMERODN': u'72396320'},
    u'_type': u'my-type'},
   {u'_id': u'IcatyW8BbundlQk4fUZB',
    u'_index': u'my-index',
    u'_score': 1.0,
    u'_source': {u'CODESTAB': u'NA',
     u'ESCMAEAGR1': u'04',
     u'IDADEMAE': u'20',
     u'NUMERODN': u'72374657'},
    u'_type': u'my-type'},
   {u'_id': u'IsatyW8BbundlQk4fUZB',
    u'_index': u'my-index',
    u'_score': 1.0,
    u'_source': {u'CODESTAB': u'2755157',
     u'ESCMAEAGR1': u'06',
     u'IDADEMAE': u'28',
     u'NUMERODN': u'7237

### busca lógica

In [75]:
content = {
    "query": {
        "bool": {
            "must": [
                {
                    "term": {
                        "IDADEMAE": 21
                    }
                },
                {
                    "term": {
                        "ESCMAEAGR1": "06"
                    }
                }
            ]
        }
    }
}

In [76]:
res = es.search(index="my-index", body=content)

In [77]:
res

{u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 1, u'total': 1},
 u'hits': {u'hits': [{u'_id': u'LMatyW8BbundlQk4fUZB',
    u'_index': u'my-index',
    u'_score': 3.6686769,
    u'_source': {u'CODESTAB': u'2777770',
     u'ESCMAEAGR1': u'06',
     u'IDADEMAE': u'21',
     u'NUMERODN': u'69726563'},
    u'_type': u'my-type'}],
  u'max_score': 3.6686769,
  u'total': {u'relation': u'eq', u'value': 1}},
 u'timed_out': False,
 u'took': 5}

### searching methods

In [46]:
def buscaExata(numerodn, codestab, idademae, escmaeagr1, startId=0): 
    
    global es
    
    content = {
        'size': 30,
        'query': {
            'bool': {
                'must': [
                    {'match_phrase': {'NUMERODN': '"' + numerodn + '"'}},
                    {'match_phrase': {'CODESTAB': '"' + codestab + '"'}}, 
                    {'match': {'IDADEMAE': idademae}},
                    {'match': {'ESCMAEAGR1': escmaeagr1}}
                ]
            }
        }
    }
    force = True
    while force:
        try:
            res = es.search(index="my-index", body=content)
            force = False
        except:
            pass
    return res['hits']['hits']

In [52]:
def buscaAproximada(numerodn, codestab, idademae, escmaeagr1, startId=0):
    
    global es
    
    content = {
        'size': 100,
        'query': {
            'bool': {
                'should': [
                    {'match': {'NUMERODN': {'query': numerodn, 'fuzziness':'AUTO', 'operator':'or', 'boost':'2'}}},
                    {'match': {'CODESTAB': {'query': codestab, 'fuzziness':'AUTO', 'operator':'or', 'boost':'2'}}},
                    {'match': {'IDADEMAE': {'query': idademae, 'fuzziness':'AUTO', 'operator':'or', 'boost':'0.5'}}},
                    {'match': {'ESCMAEAGR1': {'query': escmaeagr1, 'fuzziness':'AUTO', 'operator':'or'}}}
                ]
            }
        }
    }
    force = True
    while force:
        try:
            res = es.search(index="my-index", body=content)
            force = False
        except:
            pass
    return res['hits']['hits']

In [53]:
buscaExata("72390242", "2786095", "25", "01")

[{u'_id': u'H8atyW8BbundlQk4fUZA',
  u'_index': u'my-index',
  u'_score': 10.55623,
  u'_source': {u'CODESTAB': u'2786095',
   u'ESCMAEAGR1': u'01',
   u'IDADEMAE': u'25',
   u'NUMERODN': u'72390242'},
  u'_type': u'my-type'}]

In [63]:
# um campo modificado (note que o retorno é vazio)
buscaExata("72390242", "2786095", "33", "01")

[]

In [57]:
buscaAproximada("72390242", "2786095", "25", "01")

[{u'_id': u'H8atyW8BbundlQk4fUZA',
  u'_index': u'my-index',
  u'_score': 9.236701,
  u'_source': {u'CODESTAB': u'2786095',
   u'ESCMAEAGR1': u'01',
   u'IDADEMAE': u'25',
   u'NUMERODN': u'72390242'},
  u'_type': u'my-type'},
 {u'_id': u'I8atyW8BbundlQk4fUZB',
  u'_index': u'my-index',
  u'_score': 5.278115,
  u'_source': {u'CODESTAB': u'7373120',
   u'ESCMAEAGR1': u'12',
   u'IDADEMAE': u'34',
   u'NUMERODN': u'72392109'},
  u'_type': u'my-type'}]

In [65]:
# # um campo modificado (mais de um candidato é retornado)
buscaAproximada("72390242", "2786095", "33", "01")

[{u'_id': u'H8atyW8BbundlQk4fUZA',
  u'_index': u'my-index',
  u'_score': 13.195287,
  u'_source': {u'CODESTAB': u'2786095',
   u'ESCMAEAGR1': u'01',
   u'IDADEMAE': u'25',
   u'NUMERODN': u'72390242'},
  u'_type': u'my-type'},
 {u'_id': u'KMatyW8BbundlQk4fUZB',
  u'_index': u'my-index',
  u'_score': 1.0641159,
  u'_source': {u'CODESTAB': u'2777770',
   u'ESCMAEAGR1': u'08',
   u'IDADEMAE': u'33',
   u'NUMERODN': u'69726416'},
  u'_type': u'my-type'},
 {u'_id': u'KcatyW8BbundlQk4fUZB',
  u'_index': u'my-index',
  u'_score': 1.0641159,
  u'_source': {u'CODESTAB': u'2777770',
   u'ESCMAEAGR1': u'06',
   u'IDADEMAE': u'33',
   u'NUMERODN': u'69726448'},
  u'_type': u'my-type'}]

In [62]:
print len(buscaAproximada("72390242", "2786095", "33", "01"))

3
